# 데이터 로딩과 저장, 파일 형식

pandas 파일 파싱 함수
- read_csv
- read_table
- read_fwf
- read_clipboard
- read_excel : XLS, XLSX
- read_hdf
- read_html
- read_json
- read_msgpack
- read_pickle
- read_sas
- read_sql : SQL 쿼리 결과를 pandas의 DataFrame 형식으로 읽어온다.
- read_stata
- read_feather

In [9]:
%%writefile example1.csv
a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Writing example1.csv


In [10]:
import pandas as pd
df = pd.read_csv('example1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
pd.read_table('example1.csv')

,"a,b,c,d,message"
0,"1,2,3,4,hello"
1,"5,6,7,8,world"
2,"9,10,11,12,foo"


In [12]:
pd.read_table('example1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
%%writefile example2.csv
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Writing example2.csv


In [14]:
pd.read_csv('example2.csv',header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
pd.read_csv('example2.csv',names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
names = ['a','b','c','d','message']
pd.read_csv('example2.csv',names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [17]:
%%writefile example_mindex.csv
key1,key2,value1,value2
one,a,1,2
one,b,1,2
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16

Writing example_mindex.csv


In [19]:
pd.read_csv('example_mindex.csv',index_col=['key1','key2'])

value1  value2
key1 key2                
one  a          1       2
     b          1       2
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [21]:
%%writefile example3.csv
# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Writing example3.csv


In [25]:
pd.read_csv('example3.csv',skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [31]:
%%writefile example4.csv
something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

Overwriting example4.csv


In [32]:
result = pd.read_csv('example4.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [33]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [36]:
sentinels = {'message':['foo','NA'],'something':['two']}

pd.read_csv('example4.csv',na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


read_csv, read_table 함수 인자
- path
- sep 또는 delimiter
- header
- index_col
- names
- skiprows
- na_values
- comment
- parse_dates
- keep_date_col
- converters
- dayfirst
- date_parser
- nrows
- iterator
- chunksize
- skip_footer
- verbose
- encoding
- squeeze
- thousands

### 6.1.1. 텍스트 파일 조금씩 읽어오기

In [37]:
pd.options.display.max_rows = 10

In [39]:
pd.read_csv('example_mindex.csv',nrows=3)

,key1,key2,value1,value2
0,one,a,1,2
1,one,b,1,2
2,one,c,5,6


### 6.1.2 데이터를 텍스트 형식으로 기록하기

In [41]:
data = pd.read_csv('example4.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [42]:
data.to_csv('exampel_out.csv')

### 6.1.3 구분자 형식 다루기

In [43]:
%%writefile example7.csv
'a','b','c'
'1','2','3'
'1','2','3'

Writing example7.csv


In [44]:
import csv
f = open('example7.csv')

reader = csv.reader(f)

In [46]:
for line in reader:
    print(line)

["'a'", "'b'", "'c'"]
["'1'", "'2'", "'3'"]
["'1'", "'2'", "'3'"]


In [47]:
with open('example7.csv') as f:
    lines = list(csv.reader(f))

In [49]:
header, values = lines[0], lines[1:]

In [50]:
data_dict = {h: v for h, v in zip(header,zip(*values))}
data_dict

{"'a'": ("'1'", "'1'"), "'b'": ("'2'", "'2'"), "'c'": ("'3'", "'3'")}

### 6.1.4 JSON 데이터

In [51]:
obj = """
{"name":"Wes",
 "places_lived":["United States","Spain","Germany"],
 "pet":null,
 "siblings":[{"name":"Scott","age":30,"pets":["Zeus","Zuko"]},
             {"name":"Katie","age":38,
              "pets":["Sixes","Stache","Cisco"]}]
}
"""

In [53]:
import json

result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [54]:
asjson = json.dumps(result) # 파이썬 객체를 json형태로
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [55]:
siblings = pd.DataFrame(result['siblings'],columns=['name','age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [ ]:
%%writefile example.json
[{"a":1,"b":2,"c":3},
 {"a":4,"b":5,"c":6},
 {"a":7,"b":8,"c":9}]

In [70]:
data = pd.read_json('example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [71]:
%%writefile example.json
[{'a':1,'b':2,'c':3},
 {'a':4,'b':5,'c':6},
 {'a':7,'b':8,'c':9}]

Overwriting example.json


In [72]:
data = pd.read_json('example.json')

ValueError: Expected object or value